In [9]:
!pip install -U datasets
!pip install -U rouge_score
!pip install -U py7zr

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

dataset = load_dataset("Samsung/samsum")
test_data = dataset['test']

summarizer = pipeline("summarization", model="tanmay2809/BART_CNN_SAMSUM", device=0)
bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
p_summarizer = pipeline("summarization", model="google/pegasus-cnn_dailymail", device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

...

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [4]:
generated_summaries = [summarizer(dialogue["dialogue"], max_length=516, truncation=True)[0]["summary_text"] for dialogue in test_data]
bart_summaries = [bart_summarizer(dialogue["dialogue"], max_length=516, truncation=True)[0]["summary_text"] for dialogue in test_data]
p_summaries = [p_summarizer(dialogue["dialogue"], max_length=516, truncation=True)[0]["summary_text"] for dialogue in test_data]
reference_summaries = [dialogue["summary"] for dialogue in test_data]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 516, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)
Your max_length is set to 516, but your input_length is only 171. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=85)
Your max_length is set to 516, but your input_length is only 195. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=97)
Your max_length is set to 516, but your input_length is only 150. Since this is a summarization task, where o

In [11]:
from rouge_score import rouge_scorer
import numpy as np

def compute_rouge_scores(reference_summaries, generated_summaries):
    """
    Computes the ROUGE-1, ROUGE-2, and ROUGE-L scores between reference and generated summaries.
    Args:
        reference_summaries (list of str): List of reference summaries.
        generated_summaries (list of str): List of generated summaries.

    Returns:
        dict: A dictionary containing the mean ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Lists to store individual ROUGE scores
    rouge1, rouge2, rougeL = [], [], []

    # Compute ROUGE scores for each reference and generated pair
    for ref, gen in zip(reference_summaries, generated_summaries):
        scores = scorer.score(ref, gen)
        rouge1.append(scores['rouge1'].fmeasure)
        rouge2.append(scores['rouge2'].fmeasure)
        rougeL.append(scores['rougeL'].fmeasure)

    # Calculate mean scores
    results = {
        "ROUGE-1": np.mean(rouge1),
        "ROUGE-2": np.mean(rouge2),
        "ROUGE-L": np.mean(rougeL)
    }

    return results

In [13]:
import pandas as pd

# Compute ROUGE scores for each set of generated summaries
scores_generated = compute_rouge_scores(reference_summaries, generated_summaries)
scores_bart = compute_rouge_scores(reference_summaries, bart_summaries)
scores_p = compute_rouge_scores(reference_summaries, p_summaries)

# Create a table for comparison
comparison_table = pd.DataFrame({
    "Model": ["FineTuned_Bart", "BART_large_cnn", " Pegasus_cnn"],
    "ROUGE-1": [scores_generated["ROUGE-1"], scores_bart["ROUGE-1"], scores_p["ROUGE-1"]],
    "ROUGE-2": [scores_generated["ROUGE-2"], scores_bart["ROUGE-2"], scores_p["ROUGE-2"]],
    "ROUGE-L": [scores_generated["ROUGE-L"], scores_bart["ROUGE-L"], scores_p["ROUGE-L"]]
})

# Print the table
print(comparison_table)

            Model   ROUGE-1   ROUGE-2   ROUGE-L
0  FineTuned_Bart  0.429487  0.214141  0.330806
1  BART_large_cnn  0.314282  0.107552  0.231210
2     Pegasus_cnn  0.295117  0.086697  0.226842


In [14]:
comparison_table.to_csv("rouge_comparison.csv", index=False)